In [2]:
import os
import subprocess
import zipfile

current_directory = os.getcwd()

## Extraction of sensor files (scenexxxx_xx.sens)

In [ ]:
scans_directory = "/cluster/project/cvg/data/scannet/scans"
output_dir = os.path.join(current_directory, "data/scans")

def run_reader(filename, output_path):
    command = [
        "python",
        "scripts/reader.py",
        "--filename",
        filename,
        "--output_path",
        output_path,
        #"--export_depth_images",
        "--export_color_images"
        #"--export_poses",
        #"--export_intrinsics"
    ]
    subprocess.run(command)

for folder in os.listdir(scans_directory):
    if folder.startswith("scene"):
        sensor_file = os.path.join(scans_directory, folder, f"{folder}.sens")
        output_path = os.path.join(output_dir, folder)
        if os.path.isfile(sensor_file):
            run_reader(sensor_file, output_path)
        else:
            print(f"Sensor file not found for {folder}")

## Extraction of 2D instance-label (scenexxxx_xx_2d-instance-filt.zip & scenexxxx_xx_2d-label-filt.zip)

### Unzip files

In [ ]:
zip_files_dir = "/cluster/project/cvg/data/scannet/scans"
output_dir = os.path.join(current_directory, "data/convert_scannet")

def unzip_file(zip_file_path, output_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

for folder in os.listdir(zip_files_dir):
    if folder.startswith("scene"):
        zip_file = os.path.join(zip_files_dir, folder, f"{folder}_2d-instance-filt.zip")
        output_path = os.path.join(output_dir, f"{folder}")
        unzip_file(zip_file, output_path)
        zip_file = os.path.join(zip_files_dir, folder, f"{folder}_2d-label-filt.zip")
        output_path = os.path.join(output_dir, f"{folder}")
        unzip_file(zip_file, output_path)

### Convert instance and label images

In [4]:
input_dir = os.path.join(current_directory, "data/convert_scannet")
map_file = os.path.join(current_directory, "data/scannetv2-labels.combined.tsv")

def run_convert_instance_image(map_file, input_img, label_img, output_img):
    command = [
        "python",
        "scripts/convert_scannet_instance_image.py",
        "--input_instance_file",
        input_img,
        "--input_label_file",
        label_img,
        "--label_map_file",
        map_file,
        "--output_file",
        output_img
    ]
    subprocess.run(command)

def run_convert_label_image(map_file, label_img, output_img):
    command = [
        "python",
        "scripts/convert_scannet_label_image.py",
        "--input_file",
        label_img,
        "--label_map_file",
        map_file,
        "--output_file",
        output_img
    ]
    subprocess.run(command)

for folder in os.listdir(input_dir):
    if folder.startswith("scene"):
        instance_dir = os.path.join(input_dir, folder, "instance-filt")
        label_dir = os.path.join(input_dir, folder, "label-filt")
        for img in os.listdir(instance_dir):
            run_convert_instance_image(map_file, os.path.join(instance_dir, img), os.path.join(label_dir, img), os.path.join(instance_dir, img))
            run_convert_label_image(map_file, os.path.join(label_dir, img), os.path.join(label_dir, img))

## Extraction of 3D instance-label (scenexxxx_xx_3d-instance-filt.zip & scenexxxx_xx_3d-label-filt.zip)

### Unzip files